In [9]:
#importing prerequisites and necessary libraries
import requests
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

In [10]:
# Fetching latitude and longitude from city name
def get_coordinates(city_name):
    base_url = "https://nominatim.openstreetmap.org/search"
    params = {"q": city_name, "format": "json", "limit": 1}
    headers = {"User-Agent": "WeatherApp/1.0"}
    
    response = requests.get(base_url, params=params, headers=headers)
    
    if response.status_code != 200 or not response.json():
        print("❌ Could not find coordinates. Please check the city name and try again.")
        return None

    data = response.json()[0]
    return float(data["lat"]), float(data["lon"])

In [11]:
# Fetching weather data
def get_weather_data(lat, lon):
    weather_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": "temperature_2m,relative_humidity_2m,wind_speed_10m",
        "timezone": "auto"
    }
    
    response = requests.get(weather_url, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print("❌ Failed to retrieve weather data.")
        return None

In [12]:
# Generating synthetic training data
np.random.seed(42)
weather_df = pd.DataFrame({
    "temperature_2m": np.random.uniform(0, 40, 500),  # Random temperatures between 0-40°C
    "relative_humidity_2m": np.random.uniform(20, 90, 500),
    "wind_speed_10m": np.random.uniform(0, 30, 500)
})

In [13]:
# Splitting features and target
X = weather_df.drop(columns=["temperature_2m"])
Y = weather_df["temperature_2m"]

In [14]:
# Train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print("\n✅ Data successfully split into training and testing sets.")


✅ Data successfully split into training and testing sets.


In [15]:
# Training the Random Forest model
model = RandomForestRegressor(n_estimators=10, max_depth=5, random_state=42)
model.fit(X_train, Y_train)

print("\n🚀 Model training complete!")


🚀 Model training complete!


In [16]:
#Function to predict temperature for a given city
def predict_temperature(city):
    coords = get_coordinates(city)

    if not coords:
        return None

    lat, lon = coords
    print(f"\n✔️ Found '{city}' at Latitude: {lat}, Longitude: {lon}")

    # Fetch and process weather data
    weather_data = get_weather_data(lat, lon)

    if weather_data:
        df = pd.DataFrame(weather_data["hourly"])

        if "temperature_2m" not in df or df.empty:
            print("⚠️ No valid weather data found.")
            return None

        df.drop(columns=["time"], inplace=True, errors="ignore")  # Remove 'time' column if exists
        latest_conditions = df.iloc[-1][["relative_humidity_2m", "wind_speed_10m"]].values.reshape(1, -1)

        # Make prediction
        predicted_temp = model.predict(latest_conditions)[0]
        print(f"\n🔮 Predicted temperature in '{city}': {predicted_temp:.2f}°C")
        return predicted_temp

    else:
        print("⚠️ No weather data available.")
        return None

In [19]:
# Taking user input and making predictions
city = input("\n🌍 Enter a city name: ").strip()
predict_temperature(city)


✔️ Found 'New York' at Latitude: 40.7127281, Longitude: -74.0060152

🔮 Predicted temperature in 'New York': 23.73°C


c:\Users\Welcome\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


np.float64(23.72618653513136)